## Purpose of this notebook is to verify the seq2seq model (without force teaching) coding 
- tested with fixed sequence length

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
import tensorflow as tf 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
import numpy as np
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense 
from keras.layers import Lambda
from keras.layers import concatenate
from keras import backend as K


##Dummy dataset creation 
- with fixed sequence length of 10

In [ ]:
cd C:\Users\ASUS\Desktop\dataset-460-40

C:\Users\ASUS\Desktop\dataset-460-40


In [ ]:
#Function to generate a sequence of integers (e.g. from 0 to 499) 
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]

In [ ]:
#n_unique -> no. of unique digits (0,1,2,3,4,,...,499) to be chosen for use in each sample of input sequence
#n_samples -> total no. of samples in dataset

#Function to return arrays of dataset for encoder input, decoder input and decoder output
def get_dataset(n_unique, n_samples):

    encoder_input = list()
    decoder_input = list()
    decoder_output = list()

    for _ in range(n_samples):

        length = 10
        #length = randint(20, 267)
        
        encoder_ip = list()
        encoder_ip_x = generate_sequence(length, n_unique) #list of x-coordinates of encoder input
        encoder_ip_y = generate_sequence(length, n_unique) #list y-coordinates of encoder input

        for i in range(length):
            coordinate_pair = (encoder_ip_x[i], encoder_ip_y[i])
            encoder_ip.append(coordinate_pair)
        while len(encoder_ip) < 10: 
          encoder_ip.append([500,500]) 

        x_sum = 0
        x_coord_running_sum = list()  #list of x_coord running sum
        for i in range(length):
            x_sum = x_sum + encoder_ip[i][0]
            x_coord_running_sum.append(x_sum)

        y_sum = 0
        y_coord_running_sum = list()  #list of y_coord running sum
        for i in range(length):
            y_sum = y_sum + encoder_ip[i][1]
            y_coord_running_sum.append(y_sum)

        decoder_op = list()
        for i in range(length): 
            if (x_coord_running_sum[i] + y_coord_running_sum[i])%2 == 0: #if sum of x and y coord is even, output is even; else output is odd
                op = 0
            else:
                op = 1
            decoder_op.append(op)
        while len(decoder_op) < 10:
          decoder_op.append(0)

        decoder_ip =  [2] + decoder_op[:-1] #decoder_ip is one time-step ahead of decoder_op

        encoder_input.append(encoder_ip)
        decoder_input.append(decoder_ip)
        decoder_output.append(decoder_op)

    X1=np.array(encoder_input)
    X2=np.array(decoder_input).reshape(n_samples,10,1)
    Y=np.array(decoder_output).reshape(n_samples,10,1)

    #One-hot-encode
    encoder_input_onehot=list()
    for i in range(X2.shape[0]):
        row=list()
        for j in range(10):
            row.append(X1[i][j][0])
            row.append(X1[i][j][1])
        encoder_input_onehot.append(row)
    encoder_input_onehot = np.array(encoder_input_onehot)
    encoder_input_onehot = to_categorical([encoder_input_onehot], num_classes=10)

    decoder_input_onehot = to_categorical([X2], num_classes=3)
    decoder_output_onehot = to_categorical([Y], num_classes=3)

    encoder_input_onehot = encoder_input_onehot.reshape(n_samples,20,10)
    decoder_input_onehot = decoder_input_onehot.reshape(n_samples,10,3)
    decoder_output_onehot = decoder_output_onehot.reshape(n_samples,10,3)

    return encoder_input_onehot, decoder_input_onehot, decoder_output_onehot
   

In [ ]:
n_unique = 10
n_samples = 100000
X1, _, Y = get_dataset(n_unique, n_samples)

In [ ]:
print(X1.shape,Y.shape)

(100000, 20, 10) (100000, 10, 3)


In [ ]:
n_unique = 10
n_samples_valid = 10000
X1_valid,_, Y_valid = get_dataset(n_unique, n_samples_valid)

In [ ]:
print(X1_valid.shape,Y_valid.shape)

(10000, 20, 10) (10000, 10, 3)


In [ ]:
#Setup decoder input data
#decoder_input_data = np.zeros((num_samples, 1, n_features))
X2 = np.zeros((n_samples, 1, 3))
X2[:, 0, 0] = 1 

# Model creation and training

In [ ]:
encoder_inputs = Input(shape=(None, 10))
encoder = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = [state_h, state_c]

decoder_inputs = Input(shape=(1, 3))
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_dense = Dense(3, activation='softmax')

all_outputs = []
inputs = decoder_inputs
for _ in range(10):
    outputs, state_h, state_c = decoder_lstm(inputs, initial_state=states)
    outputs = decoder_dense(outputs)
    all_outputs.append(outputs)
    inputs = outputs
    states = [state_h, state_c]

decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit([X1, X2], Y, batch_size=32, epochs=100)

Epoch 1/100
3125/3125 [==============================] - 67s 19ms/step - loss: 0.6953 - accuracy: 0.4997
Epoch 2/100
3125/3125 [==============================] - 60s 19ms/step - loss: 0.6937 - accuracy: 0.4997
Epoch 3/100
3125/3125 [==============================] - 62s 20ms/step - loss: 0.6935 - accuracy: 0.49970s - loss: 0.6935 - 
Epoch 4/100
3125/3125 [==============================] - 63s 20ms/step - loss: 0.6934 - accuracy: 0.5002
Epoch 5/100
3125/3125 [==============================] - 67s 22ms/step - loss: 0.6934 - accuracy: 0.5006
Epoch 6/100
3125/3125 [==============================] - 69s 22ms/step - loss: 0.6933 - accuracy: 0.5010
Epoch 7/100
3125/3125 [==============================] - 64s 20ms/step - loss: 0.6933 - accuracy: 0.5003
Epoch 8/100
3125/3125 [==============================] - 65s 21ms/step - loss: 0.6933 - accuracy: 0.5013
Epoch 9/100
3125/3125 [==============================] - 67s 21ms/step - loss: 0.6932 - accuracy: 0.5006
Epoch 10/100
3125/3125 [===========

## Evaluate with test data

In [ ]:
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

In [ ]:
n_unique = 10
n_samples = 1
X1_test,_, y_test = get_dataset(n_unique, n_samples)

In [ ]:
X2_test = np.zeros((n_samples, 1, 3))
X2_test[:, 0, 0] = 1 

In [ ]:
prediction = model.predict([X1_test, X2_test])

In [ ]:
prediction = prediction[0]

In [ ]:
prediction = one_hot_decode(prediction)

In [ ]:
prediction

[0, 0, 1, 0, 1, 1, 1, 1, 1, 0]

In [ ]:
truth = one_hot_decode(y_test[0])

In [ ]:
truth

[0, 0, 1, 0, 1, 1, 1, 1, 1, 0]